In [ ]:
from copy import deepcopy
import json
import os
from typing import List
import re
import pandas as pd

In [ ]:
def drop_duplicates(ls: List) -> List:
    return list(filter(None, set(ls)))

def flatten(ls: List) -> List:
    return [item for sublist in ls for item in sublist]

def remove(ls: List, to_remove: List) -> List:
    ls_copy = deepcopy(ls)
    for el in to_remove:
        if el in ls_copy:
            ls_copy.remove(el)
    return ls_copy

In [ ]:
def load_perguntas():
    df = pd.read_excel("results/Perguntas.xlsx", sheet_name="finais")
    df = df.dropna(subset=["Resposta"])
    subs = {
        "Pergunta": "pergunta",
        "Resposta": "resposta",
        "Intenção": "intent",
        "Rótulos": "rotulos",
        "Modificador": "modificador",
        "Substantivo": "substantivo",
        "Recipiente": "recipiente",
        "Elocuções": "exemplos",
    }
    cols = list(subs.keys())
    df = df[cols]
    df = df.rename(columns=subs)
    df = df.fillna("")
    return df

In [ ]:
def save_skill(skill_path, skill):
    skill_root, skill_ext = os.path.splitext(skill_path)
    new_skill_path = f"{skill_root}2{skill_ext}"
    with open(new_skill_path, "w", encoding="utf-8") as f:
        json.dump(skill, f, ensure_ascii=False)

def load_skill(skill_path):
    with open(skill_path, "r", encoding="utf-8") as f:
        skill = json.load(f)
    return skill

In [ ]:
def get_intents(df):
    subset = ["intent", "pergunta", "exemplos"]
    records = df[subset].to_dict(orient="records")
    intents = [
        {
            "intent": record["intent"],
            "examples": [
                {"text": record["pergunta"]},
                *[{"text": exemplo} for exemplo in record["exemplos"].split("--")],
            ],
            "description": "",
        }
        for record in records
    ]
    return intents


In [ ]:
def get_entities(df):
    subset = ["rotulos", "modificador", "substantivo", "recipiente"]
    entities = []
    for col in subset:
        records = df[col].drop_duplicates().to_list()
        records = [r.split("-") for r in records]
        records = flatten(records)
        records = drop_duplicates(records)

        values = [
            {"type": "synonyms", "value": record, "synonyms": []} for record in records
        ]
        entity = {"entity": col, "values": values, "fuzzy_match": True}
        entities.append(entity)
    return entities


In [ ]:
def get_synonyms(a, b):
    """
    Pega sinônimos que foram adicionados a entidades através da interface do Watson
    e copia para as entidades geradas automaticamente da planilha.
    """
    ents_old = deepcopy(a)
    ents_new = deepcopy(b)
    for col in ["modificador", "rotulos", "substantivo", "recipiente"]:
        try:
            ent_old = next(ent for ent in ents_old if ent["entity"] == col)
            ent_new = next(ent for ent in ents_new if ent["entity"] == col)
        except StopIteration:
            continue
        for v_old in ent_old["values"]:
            for v_new in ent_new["values"]:
                if v_new["value"] == v_old["value"]:
                    v_new["synonyms"] = v_old["synonyms"]
    return ents_new

In [ ]:
def get_condition_string(js):
    modificador = js["modificador"]
    substantivo = js["substantivo"]
    recipiente = js["recipiente"]
    contextos = js["rotulos"].split("-") + [js["rotulos"]]
    contextos = drop_duplicates(contextos)
    rotulos_nao_contextuais = [
        "fauna",
        "flora",
        "extra",
        "fisica",
        "simbolo",
        "turismo",
        "engenharia",
        "saude",
        "geologia",
    ]
    contextos_filtrados = []
    for contexto in contextos:
        if all([rot not in contexto for rot in rotulos_nao_contextuais]):
            contextos_filtrados.append(contexto)

    if contextos_filtrados:
        conds_adicionais = [
            [
                " && ".join(
                    [
                        f"@modificador:{modificador}",
                        (f"@substantivo=={substantivo}" if substantivo else ""),
                        (f"@recipiente=={recipiente}" if recipiente else ""),
                        f"$contexto={contexto}",
                    ]
                ),
                " && ".join(
                    [
                        f"@modificador:{modificador}",
                        (f"@substantivo=={substantivo}" if substantivo else ""),
                        (f"@recipiente=={recipiente}" if recipiente else ""),
                        f"@rotulo={contexto}",
                    ]
                ),
            ]
            for contexto in contextos_filtrados
        ]
    else:
        conds_adicionais = [
            [
                " && ".join(
                    [
                        f"@modificador:{modificador}",
                        (f"@substantivo=={substantivo}" if substantivo else ""),
                        (f"@recipiente=={recipiente}" if recipiente else ""),
                    ]
                )
            ]
        ]

    conds_adicionais = list(
        map(
            lambda x: map(lambda y: re.sub(r"(&  &)| && $", "", y), x), conds_adicionais
        )
    )
    conds = [f"#{js['intent']}"] + flatten(conds_adicionais)
    cond_str = " || ".join(conds)
    return cond_str


def get_dialog_nodes(df):
    records = df.to_dict(orient="records")
    dialog_nodes = [
        {
            "type": "standard",
            "output": {
                "generic": [
                    {
                        "values": [{"text": record["resposta"]}],
                        "response_type": "text",
                        "selection_policy": "sequential",
                    }
                ]
            },
            "context": {"contexto": record["rotulos"]},
            "conditions": get_condition_string(record),
        }
        for record in records
    ]
    return dialog_nodes

In [ ]:
df = load_perguntas()
df[~(df["exemplos"]=="")]

In [ ]:
skill_path = "results/skill-Amazônia-Azul.json"
skill = load_skill(skill_path)

In [ ]:
new_intents = get_intents(df)
new_intents

In [ ]:
new_entities = get_entities(df)
new_entities

In [ ]:
new_entities = get_synonyms(skill["entities"], new_entities)
new_entities

In [ ]:
new_dialog_nodes = get_dialog_nodes(df)
new_dialog_nodes

In [ ]:
save_skill(skill_path, skill)